In [23]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.utils import save_image
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [24]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [25]:
# Updated Data Loading and Preprocessing
class SignatureDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = []
        self.labels = []
        
        for class_name in self.classes:
            class_path = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_path):
                self.images.append(os.path.join(class_path, img_name))
                self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        if self.transform:
            image = self.transform(image)
        return image, label

In [26]:
# Data Augmentation
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
])

# Load and split the dataset
def load_and_split_data(root_dir, train_ratio=0.8):
    full_dataset = SignatureDataset(root_dir=root_dir, transform=transform)
    train_size = int(train_ratio * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
    return train_dataset, test_dataset

In [27]:
# VAE Implementation (unchanged)
class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.ReLU(),
            nn.Flatten()
        )
        
        self.fc_mu = nn.Linear(256 * 4 * 4, latent_dim)
        self.fc_logvar = nn.Linear(256 * 4 * 4, latent_dim)
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256 * 4 * 4),
            nn.ReLU(),
            nn.Unflatten(1, (256, 4, 4)),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Sigmoid()
        )
        
    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        return self.decoder(z)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [28]:
# GAN Implementation (unchanged)
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256 * 4 * 4),
            nn.ReLU(),
            nn.Unflatten(1, (256, 4, 4)),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Tanh()
        )
    
    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

In [29]:
# Training functions (slightly modified)
def train_vae(vae, optimizer, train_loader, num_epochs=100):
    vae.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch, _ in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = vae(batch)
            loss = vae_loss(recon_batch, batch, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')

def train_gan(generator, discriminator, g_optimizer, d_optimizer, train_loader, num_epochs=100):
    criterion = nn.BCELoss()
    for epoch in range(num_epochs):
        for batch, _ in train_loader:
            batch = batch.to(device)
            batch_size = batch.size(0)
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            # Train Discriminator
            d_optimizer.zero_grad()
            outputs = discriminator(batch)
            d_loss_real = criterion(outputs, real_labels)
            d_loss_real.backward()

            noise = torch.randn(batch_size, generator.latent_dim).to(device)
            fake_images = generator(noise)
            outputs = discriminator(fake_images.detach())
            d_loss_fake = criterion(outputs, fake_labels)
            d_loss_fake.backward()
            d_loss = d_loss_real + d_loss_fake
            d_optimizer.step()

            # Train Generator
            g_optimizer.zero_grad()
            outputs = discriminator(fake_images)
            g_loss = criterion(outputs, real_labels)
            g_loss.backward()
            g_optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')

def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [30]:
# Generating New Signatures (updated for group-based generation)
def generate_vae_signatures(vae, num_samples=10, group_idx=None):
    vae.eval()
    with torch.no_grad():
        if group_idx is not None:
            # Implement group-specific latent space sampling
            z = torch.randn(num_samples, vae.latent_dim).to(device) + group_idx
        else:
            z = torch.randn(num_samples, vae.latent_dim).to(device)
        samples = vae.decode(z)
    return samples

def generate_gan_signatures(generator, num_samples=10, group_idx=None):
    generator.eval()
    with torch.no_grad():
        noise = torch.randn(num_samples, generator.latent_dim).to(device)
        if group_idx is not None:
            z = noise + group_idx
        else:
            z = noise
        samples = generator(noise)
    return samples

# Metrics and Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch, _ in test_loader:
            batch = batch.to(device)
            recon_batch, mu, logvar = model(batch)
            loss = vae_loss(recon_batch, batch, mu, logvar)
            total_loss += loss.item()
    return total_loss / len(test_loader)

In [31]:
latent_dim = 100
root_dir = 'Grouped_Output'  

In [32]:
# Load and split the data
train_dataset, test_dataset = load_and_split_data(root_dir)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)   

In [33]:
# VAE training
vae = VAE(latent_dim).to(device)
vae_optimizer = optim.Adam(vae.parameters())
train_vae(vae, vae_optimizer, train_loader,num_epochs=1500)

Epoch 1, Loss: 160486.9395
Epoch 2, Loss: 159183.2805
Epoch 3, Loss: 158418.9404
Epoch 4, Loss: 158000.5315
Epoch 5, Loss: 157556.5911
Epoch 6, Loss: 157072.8682
Epoch 7, Loss: 156664.5374
Epoch 8, Loss: 156205.1943
Epoch 9, Loss: 156036.9700
Epoch 10, Loss: 154687.1777
Epoch 11, Loss: 151826.0679
Epoch 12, Loss: 149162.1990
Epoch 13, Loss: 148186.1682
Epoch 14, Loss: 146580.0588
Epoch 15, Loss: 144748.9956
Epoch 16, Loss: 143969.8638
Epoch 17, Loss: 144594.3503
Epoch 18, Loss: 143624.7383
Epoch 19, Loss: 143081.6475
Epoch 20, Loss: 143026.6829
Epoch 21, Loss: 143091.7192
Epoch 22, Loss: 143065.9795
Epoch 23, Loss: 142641.6670
Epoch 24, Loss: 142886.9382
Epoch 25, Loss: 143510.9226
Epoch 26, Loss: 144007.5325
Epoch 27, Loss: 142485.5918
Epoch 28, Loss: 142469.4329
Epoch 29, Loss: 141973.4805
Epoch 30, Loss: 141908.4458
Epoch 31, Loss: 142476.3782
Epoch 32, Loss: 141495.2256
Epoch 33, Loss: 142145.1575
Epoch 34, Loss: 141791.3718
Epoch 35, Loss: 142652.3079
Epoch 36, Loss: 142474.3604
E

In [34]:
# GAN training
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
train_gan(generator, discriminator, g_optimizer, d_optimizer, train_loader,num_epochs=2500)

Epoch [1/2500], d_loss: 0.1563, g_loss: 3.6282
Epoch [2/2500], d_loss: 0.0669, g_loss: 4.6441
Epoch [3/2500], d_loss: 0.0234, g_loss: 5.3593
Epoch [4/2500], d_loss: 0.0183, g_loss: 5.7371
Epoch [5/2500], d_loss: 0.0136, g_loss: 6.2044
Epoch [6/2500], d_loss: 0.0110, g_loss: 6.3692
Epoch [7/2500], d_loss: 0.0075, g_loss: 6.5838
Epoch [8/2500], d_loss: 0.0104, g_loss: 7.0160
Epoch [9/2500], d_loss: 0.0061, g_loss: 7.0522
Epoch [10/2500], d_loss: 0.0111, g_loss: 7.5741
Epoch [11/2500], d_loss: 0.0220, g_loss: 10.8567
Epoch [12/2500], d_loss: 0.9823, g_loss: 10.5954
Epoch [13/2500], d_loss: 0.1170, g_loss: 9.6467
Epoch [14/2500], d_loss: 1.0300, g_loss: 13.2679
Epoch [15/2500], d_loss: 0.3403, g_loss: 4.9228
Epoch [16/2500], d_loss: 0.2259, g_loss: 5.9887
Epoch [17/2500], d_loss: 0.1600, g_loss: 5.6613
Epoch [18/2500], d_loss: 0.0983, g_loss: 6.3546
Epoch [19/2500], d_loss: 0.2276, g_loss: 9.2805
Epoch [20/2500], d_loss: 0.5649, g_loss: 10.0288
Epoch [21/2500], d_loss: 0.0252, g_loss: 5.53

In [35]:
# Generate new signatures
vae_samples = generate_vae_signatures(vae)
gan_samples = generate_gan_signatures(generator)

In [36]:
# Save generated samples
save_image(vae_samples, 'Output/vae_generated_signatures.png')
save_image(gan_samples, 'Output/gan_generated_signatures.png')

In [37]:
# Evaluate models
vae_loss = evaluate_model(vae, test_loader)
print(f'VAE Test Loss: {vae_loss:.4f}')

VAE Test Loss: 139036.5859


In [38]:
# Generate group-specific signatures (placeholder)
num_groups = len(train_dataset.dataset.classes)
for group_idx in range(num_groups):
    group_name = train_dataset.dataset.classes[group_idx]
    vae_group_samples = generate_vae_signatures(vae, num_samples=1, group_idx=group_idx)
    gan_group_samples = generate_gan_signatures(generator, num_samples=1, group_idx=group_idx)
    os.makedirs("Output/"+group_name, exist_ok=True)
    save_image(vae_group_samples, f'Output/{group_name}/vae_generated_signatures_group_{group_name}.png')
    save_image(gan_group_samples, f'Output/{group_name}/gan_generated_signatures_group_{group_name}.png')

print("Training, evaluation, and group-specific generation complete!")

Training, evaluation, and group-specific generation complete!
